<a href="https://colab.research.google.com/github/toby-p/w266-final-project/blob/main/Amazon_product_data_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet

In [13]:
import pandas as pd
import pickle
import random
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertForSequenceClassification 

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-07-19 03:19:47.074301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 03:19:47.075066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 03:19:47.075674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 03:19:47.076445: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 03:19:47.077085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

True

3823 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


In [3]:
# Code for helping save models locally after training:

import datetime
import os

# Directory where models will be stored in GDrive:
MODEL_DIR = os.path.join(os.path.expanduser("~"), "models")

# Make the directories for storing results if they don't exist yet:
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)


def local_save_dir(*subdir: str, model_name: str = "test_model"): 
    """Create timestamped directory local for storing checkpoints or models.
    
    Args:
        subdir: optional subdirectories of the main model directory
            (e.g. `checkpoints`, `final_model`, etc.)
        model_name: main name for directory specifying the model being saved.
    """
    model_dir = f"{MODEL_DIR}/{model_name}"
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    for s in subdir:
        model_dir = f"{model_dir}/{s}"
        if not os.path.exists(model_dir):
            os.mkdir(model_dir)
    now = datetime.datetime.now()
    now_str = now.strftime("%Y_%m_%d__%H_%M_%S")
    dir_path = f"{model_dir}/{now_str}"
    os.mkdir(dir_path)
    print(f"Created dir: {dir_path}")
    return dir_path


local_save_dir("checkpoints", model_name = "test_model")

Created dir: /home/tp/models/test_model/checkpoints/2022_07_19__03_19_47


'/home/tp/models/test_model/checkpoints/2022_07_19__03_19_47'

## Create train and test data

In [4]:
# Using the datasets created in a separate notebook and saved to Github:
train_url = "https://raw.githubusercontent.com/toby-p/w266-final-project/main/data/amazon/train.csv"
test_url = "https://raw.githubusercontent.com/toby-p/w266-final-project/main/data/amazon/test.csv"
val_url = "https://raw.githubusercontent.com/toby-p/w266-final-project/main/data/amazon/val.csv"


def shuffle(df: pd.DataFrame):
    "Make sure data is shuffled (deterministically)."
    ix = list(df.index)
    random.seed(42)
    random.shuffle(ix)
    return df.loc[ix].reset_index(drop=True)


amazon_train = shuffle(pd.read_csv(train_url, encoding="latin1"))
amazon_test = shuffle(pd.read_csv(test_url, encoding="latin1"))
amazon_val = shuffle(pd.read_csv(val_url, encoding="latin1"))

In [5]:
amazon_train.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_numerator,helpful_denominator,helpful_category,helpful_ratio,label
47995,A3ISCCD1ON4O9E,1606991612,Jed Palmer,"[0, 0]",Great historic comic read. Still full of fun a...,5.0,Captain Easy - an easy read,1384646400,"11 17, 2013",0,0,unhelpful,NaN,0
47996,A5EOOYAS6VRO9,B00GZ8MHNO,steve,"[0, 0]",This book tell the story about Marilyn Monroe ...,4.0,marilyn monroe grow up,1388275200,"12 29, 2013",0,0,unhelpful,NaN,0
47997,A3NW9Y963QZ8WI,B00B6BRCL2,Lorraine,"[0, 0]",No real in-depth story here. It seemed to be ...,2.0,ok,1395705600,"03 25, 2014",0,0,unhelpful,NaN,0
47998,AFWFBMKH8O76X,0399152008,ann strouhal From: Ann Strouhal/ Tx,"[0, 0]",C.J.Box has out done himself with this novel. ...,5.0,Trophy Hunt by C.J. Box: A seat of the edge st...,1398384000,"04 25, 2014",0,0,unhelpful,NaN,0
47999,A1HR0IK78TXDEB,0316211222,Jo Thomas,"[0, 0]",Enjoyed it from start to finish. Keeps you gue...,5.0,Another good James Patterson book,1389916800,"01 17, 2014",0,0,unhelpful,NaN,0


In [6]:
amazon_test.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_numerator,helpful_denominator,helpful_category,helpful_ratio,label
5995,A1623N94C3XATU,006171447X,"Rick Shaq Goldstein ""*SHAQ*""","[7, 8]","**ON SATURDAY, DECEMBER 13, 2003 AT 8:30 PM SA...",5.0,"RICK ""SHAQ"" GOLDSTEIN SAYS: ""THE MAN WHO HELPE...",1228348800,"12 4, 2008",7,8,helpful,0.875000,1
5996,ADFYFTNZW0IQM,0312095511,Lon Dee,"[9, 13]",This was a very fun book to read. It's not a l...,4.0,Humorous and Entertaining,1084752000,"05 17, 2004",9,13,helpful,0.692308,1
5997,A51KNMQWHGPMB,1439176191,"FRANKLY FRANK ""FRANK""","[7, 40]",CHENEY GOT TO WHERE HE WAS BECAUSE HE KNOWS HO...,1.0,REVISIONIST HISTORY THRU CHENEY'S EYES,1316908800,"09 25, 2011",7,40,helpful,0.175000,1
5998,A28MHD2DDY6DXB,1442367547,"Allison A. Slater ""Gryphon50""","[17, 25]",I love to read but I tend to start a lot of bo...,5.0,amazing book,1378771200,"09 10, 2013",17,25,helpful,0.680000,1
5999,A21VNCWPT9YGCK,0936348070,Karla Skinner,"[15, 17]","Recommended by a contact on a survival board, ...",5.0,One of the best.,1168300800,"01 9, 2007",15,17,helpful,0.882353,1


In [7]:
amazon_val.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,helpful_numerator,helpful_denominator,helpful_category,helpful_ratio,label
5995,A2NV8ECUE5QPAW,B00FCKGFMU,Chris Cennedy,"[4, 14]",The writing is so bad in this book that I coul...,1.0,Unreadable,1380240000,"09 27, 2013",4,14,helpful,0.285714,1
5996,A680RUE1FDO8B,0816680817,Jerry Saperstein,"[35, 37]",Michael Schumacher is a gem. His writing style...,5.0,Superb reportage and history,1134345600,"12 12, 2005",35,37,helpful,0.945946,1
5997,A35CC2LV7L5GFE,0991127390,Jaime @FTLOBbyJBlog,"[5, 5]",5 StarsI feel like this is where I apologize a...,5.0,Get your tissue ready!!!,1393977600,"03 5, 2014",5,5,helpful,1.000000,1
5998,AWEJ5JKD3MQMU,0878339280,"Timothy J. Bazzett ""BookHappy""","[0, 0]",Reading a book about major league baseball sho...,5.0,"Magical memories of a very special year. Go, T...",1398729600,"04 29, 2014",0,0,unhelpful,NaN,0
5999,A2OWTQLTN2ND2F,0007447868,Bookworm the villain,"[7, 11]","Wow, how can a great writer have such a bad bo...",1.0,unreadable,1285200000,"09 23, 2010",7,11,helpful,0.636364,1


In [8]:
x_train = amazon_train["reviewText"]
y_train = amazon_train["label"]
x_val = amazon_val["reviewText"]
y_val = amazon_val["label"]
x_test = amazon_test["reviewText"]
y_test = amazon_test["label"]

In [9]:
print(f"Shape x_train: {x_train.shape}")
print(f"Shape x_val: {x_val.shape}")
print(f"Shape x_test: {x_test.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape y_val: {y_val.shape}")
print(f"Shape y_test: {y_test.shape}")

Shape x_train: (48000,)
Shape x_val: (6000,)
Shape x_test: (6000,)
Shape y_train: (48000,)
Shape y_val: (6000,)
Shape y_test: (6000,)


## Tokenize inputs

In [12]:
# Using BERT base uncased tokenizer as per the paper:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Use sequence length 320, which achieved best accuracy and F1-score of all sequence lengths tried in the paper:
# https://link.springer.com/article/10.1007/s10660-022-09560-w/tables/4
max_length = 320

train_encodings = bert_tokenizer(
    list(x_train.values), 
    max_length=max_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf'
)

valid_encodings = bert_tokenizer(
    list(x_val.values), 
    max_length=max_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf'
)

test_encodings = bert_tokenizer(
    list(x_test.values), 
    max_length=max_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf'
)

## Model 1: Baseline Amazon product data fine-tuned model

* All layers unfrozen;
* Same parameters as best Bilal baseline model;
* Trained for 4 epochs.

In [27]:

MODEL_NAME = "amazon_finetune"


def amazon_finetune():
    """Create a BERT model using the model and parameters specified in the Bilal paper:
    https://link.springer.com/article/10.1007/s10660-022-09560-w/tables/2

        - model: TFBertForSequenceClassification
        - learning rate: 2e-5
        - epsilon: 1e-8
    """
    # Using the TFBertForSequenceClassification as specified in the paper:
    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Don't freeze any layers:
    untrainable = []
    trainable = [w.name for w in bert_model.weights]

    for w in bert_model.weights:
        if w.name in untrainable:
            w._trainable = False
        elif w.name in trainable:
            w._trainable = True

    # Compile the model:
    bert_model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]
    )

    return bert_model


model = amazon_finetune()
print(model.summary())

# Train the model using the specifications from the paper: https://link.springer.com/article/10.1007/s10660-022-09560-w/tables/2
# -- epochs = 4
# -- batch_size = 32

# Create directory for storing checkpoints after each epoch:
checkpoint_dir = local_save_dir("checkpoints", model_name = MODEL_NAME)
checkpoint_path = checkpoint_dir + "/cp-{epoch:04d}.ckpt"

# Create a callback that saves the model's weights:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1)

# Fit the model saving weights every epoch:
history = model.fit(
    [train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
    y_train.values,
    validation_data=(
        [valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask], 
        y_val.values
        ),
    batch_size=32, 
    epochs=4,
    callbacks=[cp_callback]
)

# Save the entire model to GDrive:
model_dir = local_save_dir("full_model", model_name = MODEL_NAME)
model.save(model_dir)

# Save scores on the test set:
test_score = model.evaluate([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask], y_test)
print("Test loss:", test_score[0])
print("Test accuracy:", test_score[1])
score_fp = os.path.join(model_dir, "test_score.txt")
with open(score_fp, "w") as f:
    f.write(f"Test loss = {test_score[0]}\n")
    f.write(f"Test accuracy = {test_score[1]}\n")

# Save predictions and classification_report:
predictions = model.predict([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask])
preds_fp = os.path.join(model_dir, "test_predictions.csv")
pred_df = pd.DataFrame(predictions.to_tuple()[0], columns=["pred_prob_0", "pred_prob_1"])
pred_df["yhat"] = pred_df[["pred_prob_0", "pred_prob_1"]].values.argmax(1)
pred_df["y"] = y_test
pred_df["category"] = np.where((pred_df["yhat"] == 1) & (pred_df["y"] == 1), "tp", None)
pred_df["category"] = np.where((pred_df["yhat"] == 0) & (pred_df["y"] == 0), "tn", pred_df["category"])
pred_df["category"] = np.where((pred_df["yhat"] == 1) & (pred_df["y"] == 0), "fp", pred_df["category"])
pred_df["category"] = np.where((pred_df["yhat"] == 0) & (pred_df["y"] == 1), "fn", pred_df["category"])
pred_df.to_csv(preds_fp, encoding="utf-8", index=False)
report = classification_report(y_test, pred_df["yhat"])
report_fp = os.path.join(model_dir, "classification_report.txt")
with open(report_fp, "w") as f:
    for line in report.split("\n"):
        f.write(f"{line}\n")
print(f"{MODEL_NAME} - test set results")
print(report)

# Save the history file:
hist_dir = local_save_dir("history", model_name = MODEL_NAME)
with open(os.path.join(hist_dir, "hist_dict"), "wb") as f:
    pickle.dump(history.history, f)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None
Created dir: /home/tp/models/amazon_finetune/checkpoints/2022_07_19__03_27_28
Epoch 1/4


2022-07-19 03:27:52.457740: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 150.00MiB (rounded to 157286400)requested by op tf_bert_for_sequence_classification/bert/encoder/layer_._9/attention/self/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-19 03:27:52.457814: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-07-19 03:27:52.457833: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 80, Chunks in use: 79. 20.0KiB allocated for chunks. 19.8KiB in use in bin. 884B client-requested in use in bin.
2022-07-19 03:27:52.457845: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_for_sequence_classification/bert/encoder/layer_._9/attention/self/Softmax' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._r